In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from newspaper import Article
import math

In [5]:
def get_links(result_link):
    """Gets all article links on a given page on a news site"""
    links=[]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
    r = requests.get(result_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    links_results = soup.findAll('a', class_='headline-link')
    
    #Add featured as well
    
    for result in links_results:
        parent = result.parent
        if parent.has_attr('class') and 'the-content' in parent['class']:
            links.append(result.get('href'))
     

    return links

def get_article(article_url):
    title = ""
    article = ""
    author = ""
    date = ""
    
    """Takes in a link to an article and returns the formatted article as text"""
    article = Article(article_url)
    article.download()
    article.parse()
    
    title = article.title
    author = article.authors
    date = article.publish_date
    article = article.text
    
        
    return title, author, date, article

In [6]:
links = get_links('https://www.climatedepot.com')

titles = []
authors = []
dates = []
articles = []

for link in links:
    
    try:
        title, author, date, article = get_article(link)
        titles.append(title)    
        authors.append(author)
        dates.append(date)
        articles.append(article)
    except:
        pass

print('Total articles: {}'.format(len(articles)))

Total articles: 43


In [24]:
authors2 = [", ".join(str(author) for author in author_list) for author_list in authors]
df = pd.DataFrame(data={'title': titles, 'author': authors2, 'date': dates, 'article': articles}).drop_duplicates(keep='first')
df['denial?'] = [1]*len(df.index)
df.head(5)

,title,author,date,article,denial?
0,New UN ‘prediction’ report timed to NYC UN sum...,Marc Morano,2019-08-29 00:00:00,UN IPCC draft report ‘to call for radical acti...,1
1,NASA: Global Wildfires Drop By 25% Since 2003 ...,Marc Morano,2019-08-29 00:00:00,News reports about the Amazon fires strike a f...,1
2,Science Goes Up In Rain Forest Smoke,,2019-08-28 13:21:34+00:00,Science Goes Up In Rain Forest Smoke\n\nIf jou...,1
3,Awesome! U.S. Navy Shut Down Climate Change Ta...,,2019-08-28 00:00:00,The U.S. Navy shut down its Task Force Climate...,1
4,Climate Change Could Leave Obama’s Possible Ne...,,2019-08-26 00:00:00,Former President Barack Obama’s possible new m...,1


In [25]:
df.to_csv('../../data/right/data_climate_depot.csv', index=False)
df.head()

,title,author,date,article,denial?
0,New UN ‘prediction’ report timed to NYC UN sum...,Marc Morano,2019-08-29 00:00:00,UN IPCC draft report ‘to call for radical acti...,1
1,NASA: Global Wildfires Drop By 25% Since 2003 ...,Marc Morano,2019-08-29 00:00:00,News reports about the Amazon fires strike a f...,1
2,Science Goes Up In Rain Forest Smoke,,2019-08-28 13:21:34+00:00,Science Goes Up In Rain Forest Smoke\n\nIf jou...,1
3,Awesome! U.S. Navy Shut Down Climate Change Ta...,,2019-08-28 00:00:00,The U.S. Navy shut down its Task Force Climate...,1
4,Climate Change Could Leave Obama’s Possible Ne...,,2019-08-26 00:00:00,Former President Barack Obama’s possible new m...,1
